# 分类问题
- 二分类
    - 如果是onehotlabel是\[10]还是\[01]
    - 或者逻辑斯蒂就是0还是1
- 多类别分类
    - 是二分类的推广
- 多标签分类
    - 比如一张图片可以同时属于多个类别
    - 5个类,一个图的onehot可能是\[11100]
- 多任务分类
    - 多标签的推广

##  分类概述
input是一个图片

output是一个高度抽象的符号

也就是高维张量输出成了低位标量/向量

## 常见数据集
- Mnist: 0-9的十分类问题
- Cifar: 彩图的多分类
- ImageNet: 1000分类
- Fashion_mnist: 衣服

现在分类的数据集用来做理论算法的验证, 而精度不重要了

## 二分类
一般不用one hot

sigmoid:
$$h_{\theta}(\vec{x}) = g(f(\vec{x}))=g(\vec{\theta}^T \cdot \vec{x}))= \frac{1}{1+e^{-\vec{\theta} ^T \vec{x}}}$$
Loss:
$$J(\theta)=-\frac{1}{m} \sum_{i=1}^{m}[y \log (h_{\theta}(x_{i}))+(1-y)\log(1-h_{\theta}(x_{i}))]$$

Pytorch封装好了:

```torch.nn.BCEWithLogitsLoss```

## 多分类
类和类必须平级

不能包含

甚至大类下面有子类, 不能使用

类和类之间易于区分

都是关于狗的,但是狗的细节有差别, 这样还没有很好的解决方法

### 结构:

图片-> n \* (卷积 -> relu -> pooling) -> flatten -> 特征向量 -> FC -> SoftMax

Loss: 交叉熵

Pytorch封装好了: 两种方法:
- ```torch.nn.CrossEntropyLoss```: 输入是softmax的结果, 输出就是loss

- 先```torch.nn.logSoftmax```, 再```torch.nn.NLLloss```
## 多标签分类:
人类蛋白质分类

一个图片可能属于多个类0001001110

简化问题:

多次二分类, 第一个是还是不是, 第二个是还是不是...

Loss:

```torch.nn.multiLabelSoftMarginLoss```

```torch.nn.multiLabelMarginLoss```

## 多任务分类

之前的问题,都是每个类别都是"是"或者"不是"两种

一张人脸图:
- 性别:
    - 男
    - 女
    - 其他
- 帽子
    - 有帽子
    - 没帽子
- 口罩
    - 有口罩
    - 没口罩
    - 其他
- 眼镜
    - 太阳镜
    - 近视镜
    - 没有眼镜
    - 其他

分类的标签不能用onehot表示, 可能表示为1023

而且数据不均衡会带来很大的问题

In [ ]:
import torch
import torchvision

In [ ]:
my_alexnet = torchvision.models.alexnet(pretrained=True)
# Linux:
# /home/a/.cache/torch/checkpoints/

In [ ]:
import os
print(os.getcwd())

In [ ]:
my_trsfms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914,0.4822,0.4465],[0.247,0.243,0.261])
])

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root=".",train=True, transform= my_trsfms,download=True)
# 下载cifar10并解压放在此目录下

In [ ]:
import cv2
import matplotlib.pyplot as plt
plt.imshow(train_dataset.data[1])

In [ ]:
test_dataset = torchvision.datasets.CIFAR10(root=".",train=False,transform=my_trsfms, download=True)

pytorch借鉴了caffe，包括：
- 数据结构:caffe叫blob, torch是tensor
- 层功能定义: layer
- 模型的结构: net
- 最优化: caffe里叫solver, torch里是train和optim

这是结构上一样，而对于数据流来说：torch:
- 首先要读取图片，作为一个numpy形式的tensor
- tensor进入内存，或者GPU显存
- 不建议直接用tensor进行操作，要归一化，减去均值除以方差
- 增广
- 变成pytorch的tensor形式
- 进入net（module）
- 开始迭代，loss重新进入网络

![20](20.png)

In [ ]:
# batchsize一般这些超参数要放在配置文件里
# dataloader能帮助我们更快地载入数据集
# 方便我们调整batchsize ,送到内存/显存里
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=16,shuffle=True)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset) # 测试集shuffle没有意义

In [ ]:
my_alexnet.parameters

迁移学习：

其他数据集训练模型

在新的数据集上使用

比如这里的alexNet，在ImageNet上训练，1000分类

训练的时候我们把这个alexnet的前面参数, 也就是特征提取部分固定不变,拿过来, 对于最后的1000的全连接, 变成10分类

固定的手段可以是lr=0

这样变化, 会产生新的权重, 这些权重没有训练过, 重新初始化训练即可

也就是使用了一个现成的backbone

如果新数据集更大, 前面的参数需要变化, 可以选择小的lr,重新学习



In [ ]:
for i in my_alexnet.parameters():
    i.requeires_grad = False

# 让前面的参数不再训练

In [ ]:
# my_alexnet.classifier[6]=torch.nn.Linear(in_features=4096,out_features=10)
# 这样做是错误的！

in_f = my_alexnet.classifier[6].in_features
my_alexnet.classifier[6]=torch.nn.Linear(in_f, out_features=10)

In [ ]:
learn_rate = 0.001
num_epoches = 10
criteria=torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(my_alexnet.classifier[6].parameters(),lr=learn_rate,momentum=0.9)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('CPU')
print(device)

In [ ]:
import time

In [ ]:
print(len(train_dataset[0]))
print(train_dataset[0][0].shape)
print(train_dataset[0][-1])

In [ ]:
start = time.time()
my_alexnet.to(device)
my_alexnet.train()
for epoch in range(num_epoches):
    print(f"epoch: {epoch}")
    for idx,(img,label) in enumerate(train_dataloader):
        images=img.to(device)
        labels=label.to(device)

        optimizer.zero_grad()
        output = my_alexnet(images)
        loss = criteria(output,labels)
        loss.backward()
        optimizer.step()
        
        if (idx+1)%100==0:
            end = time.time()
            print(f"index: {idx}, current loss = {loss.item()}, time: {end-start}")


In [ ]:
# eval/test
# 因为有标签,要验证
my_alexnet.to(device)
my_alexnet.eval() # my_alexnet.train(False) # 两者等价

# eval和train参数更新方式不一样

total = 0
correct = 0

for image, label in test_dataloader:
    
    image_gpu = image.to(device)
    label_gpu = label.to(device)

    output = my_alexnet(image_gpu)
    # print(output) # tensor: 2.6511, -0.8053, -1.3062,  0.1148,  0.1523,  0.9770, -2.3091, -0.5917, 0.0294, -0.1196
    # 这样不行 我们需要最大值
    correct += 1 if  torch.max(output, 1)[-1].to('cpu').to('cp=label else rrect)
    total +=1
    print(correct/total)


## 多任务分类
本身是个多标签分类的推广

把每个类别对应一个任务

那这个任务可能都不是分类问题 

比如年龄 0-100, 回归问题

这个叫多任务学习(MTL Multi Task Learning)

变成多个任务会出现问题:
- 数据不均衡
    - 取数据的时候做一些纠正, 比如表情少, 头发多 , 可以复制重复的,增广, 多去少的, 少去多的
    - GAN  对抗学习
 
- 通常情况下, loss回传回去,训练出的网络, 更倾向于原来的数据

- 类别a >> 大于类别b , 我们可对loss进行加权, 对于a的loss系数小, b的系数大, 这个叫加权CrossEntropyLoss

### Weighted Cross Entropy Loss
比如二分类
$$Loss = -y \log(P) - (1-y) \log(P)$$
$$ Weighted Loss = -\alpha y \log(P) - (1-\alpha) (1-y) \log(P)$$

这样看似可以解决正负不均衡的问题,  但是 ,  

类别\训练|难|易|
---|---|---|
正a|0.6|0.99|
负b|0.49|0.01|

0.49意味着很难分辨

意思是对于网络来说, 通过WCE能够辨别出是正样本还是负样本

但是在难和易之间也是需要不平衡的

因为如果很多样本是很困难的

很少样本是容易的

这种情况下没有辨别能力

容易的多困难的少,这样不行

困难的多容易的少,这是OK的


这种难易的不均衡通过WCE无法解决

正难>负难>正易>负易

### Focal Cross Entropy Loss

$$FCEL = -(1-P)^\gamma \log(P) - P^\gamma \log(1-P)$$

当P(y=1)=0.9, \gamma = 2

可以对容易的样本做一个限制

缺点: 过于关注难的样本.. 难的样本有可能是错的数据, 

解决: 难样本上产生过大梯度, 把它正则约束一下(gradient harmonize mechanism)
### 两者结合

$$ FocalLossWithWeight = -\alpha (1-P)^\gamma \log(P) - (1-\alpha)P^\gamma \log(1-P)$$

### 多任务的学习策略

一个Backbone 对应多个branches


每个分支任务,或者叫功能会产生loss


Loss最终相加, 考虑数量级 ,需要考虑权重
![10](10.png)

图中两种loss方式,一个最终相加

一个分别计算

## Fine-Grained Classification

细分类 

比如 植物什么种类的,花朵什么花

## Center Loss

人脸领域用的也很多

细分类其实是对图片本身做更多的标签

比如花蕊, 花瓣的形状, 花的颜色

又比如车品牌识别

目前还比较困难, 不是很好


### 分类的流程

输入图片

经过卷积等Backbone, 得到一个deeply learned features

这个features比如说是向量vector

vector经过我们的FC等操作

得到预测标签

利用LossFunction和真实标签做Loss

![3.png](3.png)

----------------

我们希望那些deeply learned features如果相互之间分离性强

也就是不同类别vector差别大,  同类别vector差别小

-------------

但是传统ML只能单纯的把两堆数据分开, 如图中xy二维坐标中红点和蓝点

但是线附近的红点和蓝点, 差别并没有那么大

蓝点凭什么跟红点靠的近呢? 那些直线附近的红点, 不应该更靠近边缘的红点吗?

也就是缺少红色和红色的距离与红色与蓝色的对比


缺少分得更开这样的细节

### 意义

出现了: 类内距离大于了类间距离
Intra-class > Inter-class

我们希望

![4](4.png)

而不是

![5](5.png)

[Center Loss](https://ydwen.github.io/papers/WenECCV16.pdf) 

Mnist 经过Backbone得到二位向量, 直接画这个二维向量:
![5](6.png)


中心部分大家相似, 周围大家相差很多

embedded表现

比如有些图片7长的像1


用传统ML在中心部分很难区分

这个是数据本身带来的

我们希望 Deeply learned features 离散型更强, 就引入center loss

## Center Loss
loss值包括两项:
- 分类结果对不对 
- 每个类中所有点到该类的中心点距离之和

这样:
- 不仅分类正确
- 而且中间的特征向量的聚集程度也提升了
也当作loss值一部分

### 具体 过程:

所有点到中心点距离之和的一半

$$L_{C} =\frac{1}{2}\sum_{i=1}^{m} ||\vec{x_{i}}-\vec{c_{y_{i}}}||_{2}^{2}$$
c_yi就是该类中心点的坐标

$$\vec{c_{y_{i}}} =\frac{1}{m} \sum_{x_{i}\in y_{i}}^{m}x_{i}$$
对于第t次到t+1次:

总loss
$$ L = L_{S} + \lambda L_{C}$$

对每个样本i, 反传梯度

$$\frac{\partial L^{t}}{\partial x_{i}^{t}}=\frac{\partial L_{S}^{t}}{\partial x_{i}^{t}} +\lambda \frac{\partial L_{C}^{t}}{\partial x_{i}^{t}}$$

更新center loss层的参数W, \mu为学习率:

$$W^{t+1} = W^{t} - \mu^{t}$$

$$\frac{\partial L^{t}}{\partial W^{t}} = W^{t}-\mu^{t}\cdot\frac{\partial L_{c}^{t}}{\partial x_{i}^{t}}$$
 
对于每个参数j, j是类别 ,更新cj


$$\vec{c_{j}}^{t+1} = \vec{c_{j}}^{t} - \alpha \Delta \vec{c_{j}} $$

\alpha 一般取0.8 ~ 0.9, 用于控制波动情况

C_yi不是固定的， 是所有点的中心:

$$\Delta \vec{c_{j}} =\frac{ \sum_{i=1}^{m} \delta(y_{i} = j)\cdot( \vec{c_{j}}-\vec{x_{i}})  }{1+\sum_{i=1}^{m} \delta(y_{i}=j)}$$

其中

$$\delta(True) =1$$

$$ \delta(False)=0$$

更新\theta_{C}

$$\theta_{C}^{t+1} =\theta_{C}^{t} -\mu^{t} \sum_{i=1}^{m}\frac{\partial L^{t}}{\partial x_{i}^{t}}\cdot\frac{\partial x_{i}^{t}}{\partial \theta_{C}^{t}}$$


然而Center Loss在Mnist作用好, 在Cifar10上差

因为Cifar同一类中样本数据距离大, 不适用

人脸, 比如人脸属于哪个类, 非常适用CenterLoss



 

- Triplet Loss
- Contrastive Loss

也非常适用于人脸

### 残差注意力网络

注意力机制:

关注点集中在可区分的特征上

比如热气球的图片,注意力不应该放在天空或者高楼上

网络的关键就是一个沙漏(Bottom-up)结构加上一个sigmoid, 得到一个0,1之间的权重值


然后权重乘以原图再+原图

![7](7.png)
经过一系列变换, 热气球确实只关注了气球本身

但是过于关注尖端部分,因此陀螺也很容易识别为气球

# 验证集精度高于训练集的问题

![8](8.png)

- 训练集数据作了增广, 预处理, 这些增广很有可能做坏了, 分布变差了, 导致loss降得慢
    - 比如一张图片增广后, 在验证集上很少找到对应的, 这张就很难训练
- 正则化模型, 比如DropOut, 训练丢弃了很多, 验证又全都要. 正则化过多会使模型有差异

- 验证的时候, 是在每个epoch结束时验证, 训练loss可以在每个bactch得到. 也就是说, 验证晚于训练, 所以显然会loss小